# Algoritmos:

### En este notebook Jupyter veremos los algoritmos que utilizamos para resolver nuestro problema:

- Máximo Descendiente

- Newton

### A continuación pasamos a la explicación de cada uno.

# Descenso por Gradiente (Gradientes descendentes):

### 1) Encabezado y librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Qué hace: importa numpy para cálculo numérico (vectores, operaciones, funciones trigonométricas) y
matplotlib.pyplot para graficar.

Breve teoría: numpy permite trabajar con arrays y operaciones vectoriales de forma eficiente; usarlo evita
bucles innecesarios y errores de precisión cuando se evalúa la función en mallas para las curvas de nivel.

### 2) Definición de la función objetivo

In [ ]:
def f(x, y):
    return 1 / (2 + np.cos(x + y)) + ((x- y)**2 + (x + y)**2) / 20

Qué hace: devuelve el valor de la función f(x,y) que vamos a minimizar.

Breve teoría: la función tiene dos componentes: 1/(2 + cos(x + y)) (término oscilatorio y acotado) y un
término cuadrático ((x−y)**2 + (x+y)**2)/20
que aporta coercividad (crece con la norma ∥(x,y)∥), lo cual favorece la
existencia de mínimos globales y proporciona curvatura para el descenso.

### 3) Gradiente analítico

In [ ]:
def grad_f(x, y):
    df_dx = np.sin(x + y) / (2 + np.cos(x + y))**2 + x / 5
    df_dy = np.sin(x + y) / (2 + np.cos(x + y))**2 + y / 5
    return np.array([df_dx, df_dy])

Qué hace: calcula las derivadas parciales ∂f/∂x y ∂f/∂y y las empaqueta en un vector.

Breve teoría: el gradiente indica la dirección de mayor aumento de f; para minimizar usamos la dirección
opuesta (−∇f). Es preferible emplear derivadas analíticas por precisión y velocidad frente a diferencias finitas.
Observa que la parte trigonométrica aporta el término sin(x + y)/(2 + cos(x + y))**2 en ambas componentes,
y la parte cuadrática da x/5 y y/5.

### 4) Parámetros del algoritmo

In [ ]:
alpha = 0.1
eps = 1e-6
max_iter = 1000


Qué hace: fija el tamaño de paso constante α, la tolerancia para parada y el número máximo de iteraciones.

Breve teoría: α condiciona estabilidad y rapidez: pasos grandes aceleran pero pueden causar divergencia;
pasos pequeños aseguran estabilidad pero hacen lenta la convergencia. ε controla la precisión del criterio de
parada (a menor ε, mayor precisión y más iteraciones).

### 5) Puntos iniciales y estructura de resultados

In [ ]:
puntos_iniciales = [np.array([2,-10]), np.array([-50, 5]), np.array([26,-10])]
resultados_grad = []

Qué hace: define varios inicios para probar la robustez del método y prepara la lista donde se guardarán
resultados.

Breve teoría: para funciones no convexas o con oscilaciones locales, distintos puntos iniciales pueden
converger a distintos mínimos locales; probar varios inicios ayuda a detectar sensibilidad y a evaluar si el
mínimo encontrado es global.

### 6) Bucle principal — inicialización por punto

In [ ]:
for x0 in puntos_iniciales:
    x = x0.copy()
    trayectoria = [x.copy()]
for i in range(max_iter):
    g = grad_f(x[0], x[1])
    x_new = x- alpha * g
    trayectoria.append(x_new.copy())
    if np.linalg.norm(x_new- x) < eps:
        break
    x = x_new

Qué hace: para cada punto inicial ejecuta el proceso iterativo del gradiente descendente:
1. calcula el gradiente g = ∇f(x),
2. actualiza x_nuevo = x − αg,
3. guarda la trayectoria,
4. comprueba si el cambio es menor que ε y, si es así, para.

Breve teoría: el criterio de parada usado es la norma del paso ∥x_{k+1}−x_k∥. Alternativamente se puede usar
∥∇f(x)∥ (norma del gradiente) para detectar cercanía a un punto crítico. Guardar la trayectoria permite
analizar comportamiento (oscilaciones, convergencia lenta/rápida).

###  7) Almacenamiento del resultado

In [ ]:
resultados_grad.append({
"inicio": x0,
"minimo": x,
"valor_minimo": f(x[0], x[1]),
"iteraciones": i+1,
"trayectoria": np.array(trayectoria)
})

Qué hace: guarda, para cada inicial, el punto final encontrado, el valor de la función en ese punto, el número
de iteraciones y la trayectoria completa.

Breve teoría: conservar estos datos permite comparar la efectividad del método según la inicialización, y
evaluar si el valor mínimo parece coincidir entre inicios (indicio de mínimo global) o difiere (múltiples mínimos
locales).

### 8) Impresión de resultados

In [ ]:
for r in resultados_grad:
    print("Punto␣inicial:", r["inicio"])
    print(" M nimo␣encontrado:", r["minimo"])
    print("Valor␣en␣el␣ m nimo:", r["valor_minimo"])
    print("Iteraciones:", r["iteraciones"])
    print("-"*50)

Qué hace: muestra por consola un resumen legible de cada experimento (inicio, mínimo, valor y número de
iteraciones).

Breve teoría: revisar estos resúmenes ayuda a detectar convergencia prematura, divergencia o que el método
quedó atrapado en un mínimo local.

### 9) Construcción de la malla y evaluación para la gráfica

In [ ]:
x_vals = np.linspace(-10, 10, 200)
y_vals = np.linspace(-10, 10, 200)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

Qué hace: crea una malla 2D en el rectángulo [−10,10]×[−10,10] y evalúa la función f en cada punto para
obtener la matriz Z de valores.

Breve teoría: las curvas de nivel ayudan a visualizar la geometría local de f: pozos, valles, crestas y cómo se
desplazan las trayectorias iterativas sobre ese paisaje.

### 10) Trazado de curvas de nivel y trayectorias

In [ ]:
plt.figure(figsize=(8,6))
plt.contour(X, Y, Z, levels=30, cmap='viridis')

for r in resultados_grad:
    T = r["trayectoria"]
    plt.plot(T[:,0], T[:,1], 'o-', label=f'Inicio {r["inicio"]}')

plt.title("Trayectorias del Método del Gradiente Descendente")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

Qué hace: dibuja las curvas de nivel de f y superpone las trayectorias de cada ejecución del gradiente
descendente. Cada punto de la trayectoria se marca y se une en orden con líneas.

Breve teoría: la visualización muestra si las trayectorias se dirigen al mismo mínimo, si oscilan (posible paso
demasiado grande) o si se estancan. También permite evaluar la eficiencia del método según la geometría
local (curvaturas grandes o pequeñas).

# Método de Newton:

### 1) Importación de librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Qué hace: carga las librerías numpy (para cálculo matricial) y matplotlib.pyplot (para graficar las trayectorias).

Teoría: el método de Newton requiere operaciones vectoriales y matriciales como la evaluación del
gradiente y la Hessiana, por lo que el uso de numpy facilita la implementación eficiente y precisa.

###  2) Definición de la función objetivo

In [ ]:
def f(x, y):
    return 1 / (2 + np.cos(x + y)) + ((x- y)**2 + (x + y)**2) / 20

Qué hace: define la función escalar f(x,y) que se desea minimizar.

Teoría: la función está compuesta por un término oscilatorio y otro cuadrático que garantiza coercividad, es decir, f(x,y) → ∞ cuando ∥(x,y)∥ → ∞.
Esto asegura la existencia de al menos un mínimo global.

###  3) Cálculo del gradiente

In [ ]:
def grad_f(x, y):
    df_dx = np.sin(x + y) / (2 + np.cos(x + y))**2 + x / 5
    df_dy = np.sin(x + y) / (2 + np.cos(x + y))**2 + y / 5
    return np.array([df_dx, df_dy])

Qué hace: calcula el gradiente ∇f(x,y) de forma analítica.

Teoría: el gradiente proporciona la dirección de máximo incremento de f; en métodos de optimización como Newton, se usa junto a la Hessiana para
determinar una dirección de búsqueda más informada que la del gradiente descendente simple.

### 4) Definición de la Hessiana

In [ ]:
def hessiana_f(x, y):
    num = np.sin(x + y)**2 + 2*np.cos(x + y) + 1
    den = (2 + np.cos(x + y))**3
    h = num / den  # término común
    return np.array([
        [h + 1/5, h],
        [h, h + 1/5]
    ])


Qué hace: implementa la matriz Hessiana exacta de la función objetivo. Definamos  u = x + y y calculaamos un término común h = g''(u) para g(u) = 1/(2 + cos u) usando la expresión del código:

h = (sin²(u) + 2·cos(u) + 1) / (2 + cos(u))³
Con ese h, construye la matriz:

H(x, y) = [[h + 1/5, h],
           [h, h + 1/5]]
donde 1/5 proviene de la parte cuadrática (x² + y²)/10 (equivalente a ((x − y)² + (x + y)²)/20).

Teoría: si escribimos f(x, y) = g(x + y) + (x² + y²)/10, la Hessiana se descompone en:

Curvatura del término oscilatorio g(x + y), que aporta h = g''(u) en la diagonal y también en los términos cruzados (fuera de la diagonal), por eso aparecen h en las cuatro entradas.
Curvatura constante de (x² + y²)/10, que añade 1/5 a cada diagonal y 0 fuera de la diagonal.


También sabemos que H es simétrica y depende solo de u = x + y.
El denominador (2 + cos u) ∈ [1, 3], por lo que no hay división por cero; aun así, valores muy pequeños de h pueden producir pasos grandes.
Cuando g''(u) > 0, H suele estar mejor condicionada y más cercana a definida positiva, lo que favorece pasos de Newton estables.




### 5) Parámetros y configuración

In [ ]:
eps = 1e-6
max_iter = 100
puntos_iniciales = [np.array([2, -10]), np.array([-50, 5]), np.array([26, -10])]

resultados_newton = []

Qué hace: establece la tolerancia ε, el número máximo de iteraciones y los puntos de inicio para probar
la robustez del método.

Teoría: el método de Newton puede converger muy rápido si el punto inicial está
cerca del mínimo, pero puede divergir si está lejos o si la Hessiana no es definida positiva. Por eso se suelen
probar distintos puntos iniciales.

### 6) Bucle principal

In [ ]:
for x0 in puntos_iniciales:
    x = x0.copy()
    trayectoria = [x.copy()]
    
    for i in range(max_iter):
        g = grad_f(x[0], x[1])
        H = hessiana_f(x[0], x[1])
        delta = np.linalg.solve(H, g)   # Resuelve H * delta = grad
        x_new = x - delta
        trayectoria.append(x_new.copy())
        
        if np.linalg.norm(x_new - x) < eps:
            break
        x = x_new

Qué hace: aplica el método de Newton en forma iterativa:
x_{k+1} = x_k − H^{-1}(x_k) ∇f(x_k)
usando la instrucción np.linalg.solve para resolver el sistema lineal en lugar de calcular explícitamente
la inversa de H.

Teoría: el método aprovecha la curvatura local para acelerar la convergencia: si H se
aproxima bien a la curvatura real de f, el método converge en muy pocas iteraciones (orden cuadrático).

### 7) Almacenamiento de resultados

In [ ]:
resultados_newton.append({
        "inicio": x0,
        "minimo": x,
        "valor_minimo": f(x[0], x[1]),
        "iteraciones": i+1,
        "trayectoria": np.array(trayectoria)
    })

Qué hace: guarda para cada ejecución los valores relevantes: punto inicial, mínimo encontrado, valor de la
función, iteraciones y trayectoria.

Teoría: este almacenamiento permite comparar convergencias y analizar
si el método alcanza el mismo punto para distintos inicios (indicador de mínimo global).

### 8) Impresión de resultados

In [ ]:
for r in resultados_newton:
    print("Punto inicial:", r["inicio"])
    print("Mínimo encontrado:", r["minimo"])
    print("Valor en el mínimo:", r["valor_minimo"])
    print("Iteraciones:", r["iteraciones"])
    print("-"*50)

Qué hace: muestra en consola la información obtenida de cada prueba.

Teoría: la revisión manual de los valores finales y el número de iteraciones ayuda a identificar posibles divergencias o estancamientos numéricos.

### 9) Construcción de la malla para graficar

In [ ]:
x_vals = np.linspace(-10, 10, 200)
y_vals = np.linspace(-10, 10, 200)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

Qué hace: crea una malla de puntos en el plano y calcula el valor de la función f en cada punto.

Teoría:
esto permite representar la superficie como curvas de nivel, útiles para visualizar la dirección y forma de la
convergencia del método.

### 10) Visualización de trayectorias

In [ ]:
plt.figure(figsize=(8,6))
plt.contour(X, Y, Z, levels=30, cmap='plasma')

for r in resultados_newton:
    T = r["trayectoria"]
    plt.plot(T[:,0], T[:,1], 'o-', label=f'Inicio {r["inicio"]}')

plt.title("Trayectorias del Método de Newton")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

Qué hace: traza las curvas de nivel de f y superpone las trayectorias obtenidas desde distintos puntos iniciales.

Teoría: visualizar las trayectorias permite comprobar la rapidez con que cada punto inicial converge
al mínimo y observar si hay sensibilidad frente al punto de partida o presencia de mínimos locales.

# Comparación entre los resultados de los códigos

In [ ]:
import pandas as pd

Se importa la librería pandas, la cual permite manejar y visualizar datos en forma tabular mediante estructuras
llamadas DataFrame. Es especialmente útil para resumir los resultados de los experimentos numéricos
y compararlos de forma ordenada.

In [ ]:
comparacion = []

Aquí se crea una lista vacía llamada 'comparacion', donde posteriormente se almacenarán los datos de interés
(como los puntos iniciales, número de iteraciones y mínimos obtenidos) para ambos métodos.

In [ ]:
for g, n in zip(resultados_grad, resultados_newton):
    comparacion.append({
        "Inicio": str(g["inicio"]),
        "Iteraciones Gradiente": g["iteraciones"],
        "Iteraciones Newton": n["iteraciones"],
        "Mínimo Gradiente": np.round(g["minimo"], 4),
        "Mínimo Newton": np.round(n["minimo"], 4)
    })

Este ciclo recorre simultáneamente las listas resultados_grad y resultados_newton, que contienen la información generada por cada método. Con zip(), se emparejan los resultados correspondientes a un mismo
punto inicial. En cada iteración se agrega un diccionario a la lista 'comparacion' con los siguientes datos:

• 'Inicio': el punto de partida desde el cual se aplicó cada método.

• 'Iteraciones Gradiente' y 'Iteraciones Newton': el número de pasos que necesitó cada algoritmo
para converger.

• 'Mínimo Gradiente' y 'Mínimo Newton': las coordenadas del punto mínimo hallado por cada método,
redondeadas a cuatro cifras decimales mediante np.round().

Este proceso permite comparar la eficiencia y la precisión de ambos algoritmos para los mismos casos iniciales.
Los datos recopilados se transforman en un DataFrame llamado 'df', lo que facilita su visualización en forma de tabla.
El comando print(df) muestra en pantalla la comparación final, donde se pueden observar las diferencias entre el número de iteraciones y los puntos óptimos obtenidos por cada método.

En resumen, este bloque de código sirve para sintetizar y analizar los resultados experimentales de ambos
algoritmos de optimización, proporcionando una forma clara de evaluar cuál de ellos converge más rápido y
hacia qué punto lo hace.